### fastText的TensorFlow实现

In [3]:
# coding=utf-8
import numpy as np
import re
import os
import codecs

import tensorflow as tf

import time
import datetime

from sklearn.utils import shuffle

#### 1.相关参数配置

In [4]:
# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("file_path", "thu_data_3class_3w", "Data source.")
tf.flags.DEFINE_integer("num_classes", 3, "number classes of datasets.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 200, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")
tf.app.flags.DEFINE_integer("num_sampled",50,"number of noise sampling") 

# Training parameters
tf.flags.DEFINE_integer("batch_size", 32, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 100, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print('\nParameters')
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


Parameters
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=32
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.1
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=200
EVALUATE_EVERY=100
FILE_PATH=thu_data_3class_3w
L2_REG_LAMBDA=0.0
LOG_DEVICE_PLACEMENT=False
NUM_CHECKPOINTS=5
NUM_CLASSES=3
NUM_EPOCHS=100



#### 2. 加载数据

#### 3. 网络模型定义

In [5]:
class fastText(object):
    def __init__(self, num_classes, sentence_length, vovab_size, embedding_dims,num_sampled,
                 l2_reg_lambda):
        """

        :param num_classes:
        :param sentence_length:
        :param vovab_size:
        :param embedding_dims:
        """
        self.num_classes = num_classes
        self.sentence_length = sentence_length
        self.vocab_size = vovab_size
        self.embedding_dims = embedding_dims
        self.l2_reg_lambda = l2_reg_lambda
        self.num_sampled = num_sampled

        self.input_x = tf.placeholder(tf.int32, [None, self.sentence_length], name='input_x')
        self.input_y = tf.placeholder(tf.int32, [None, self.num_classes], name='input_y')

        self.loss = tf.constant(0.0)
        self.network()

    def network(self):
        """

        :return:
        """
        # 1. embedding layer
        with tf.device('/cpu:0'), tf.name_scope('embedding'):
            self.Embedding = tf.Variable(tf.random_uniform([self.vocab_size, self.embedding_dims],
                                                           -1., 1.,dtype=tf.float32), name='Embedding')
            self.embedding_chars = tf.nn.embedding_lookup(self.Embedding, self.input_x)

        # 2. skip gram
        with tf.name_scope('skip-gram'):
            self.sentence_embedd = tf.reduce_mean(self.embedding_chars, axis=1)

            #FC
            self.W = tf.get_variable('W', [self.embedding_dims, self.num_classes])
            self.b = tf.get_variable('b', [self.num_classes])

            self.logits = tf.matmul(self.sentence_embedd, self.W) + self.b

        with tf.name_scope('loss'):
            # classes = tf.reshape(self.num_classes, [-1])
            # classes = tf.expand_dims(classes, 1)
            # self.loss = tf.reduce_mean(tf.nn.nce_loss(weights=tf.transpose(self.W),
            #                                       biases=self.b,
            #                                       labels=classes,
            #                                       inputs=self.sentence_embedd,
            #                                       num_sampled=self.num_sampled,
            #                                       num_classes=self.num_classes,
            #                                       partition_strategy='div'))
            self.loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits,
                                                                       labels=self.input_y)
            l2_loss = tf.nn.l2_loss(self.W)
            l2_loss += tf.nn.l2_loss(self.b)
            self.loss += l2_loss * self.l2_reg_lambda

        with tf.name_scope('accuracy'):
            self.predicted = tf.argmax(self.logits, axis=1, name='predicted')
            corr = tf.equal(self.predicted, self.input_y)
            self.accuracy = tf.reduce_mean(tf.cast(corr, dtype=tf.float32), name='accuracy')

#### 4. 模型训练

In [6]:
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # 构建cnn 节点
        ft = fastText(
            num_classes=FLAGS.num_classes,
            sentence_length=x_train.shape[1],
            vovab_size=len(word_index),
            embedding_dims=FLAGS.embedding_dims,
            num_sampled=FLAGS.num_sampled,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        # 优化算法
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)

        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name),
                                                     tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", ft.loss)
        acc_summary = tf.summary.scalar("accuracy", ft.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              ft.input_x: x_batch,
              ft.input_y: y_batch,
            }
            # 执行 节点操作
            _, step, summaries, loss, accuracy, predicted = sess.run(
                [train_op, global_step, train_summary_op, ft.loss, ft.accuracy, ft.predicted],
                feed_dict)

            time_str = datetime.datetime.now().isoformat()
            if step % 20 == 0:
                print("{}: step {}, loss {:g}, acc {:g}, predicted {:g}".format(time_str, step, loss, accuracy, predicted))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              ft.input_x: x_batch,
              ft.input_y: y_batch,
            }
            step, summaries, loss, accuracy, predicted = sess.run(
                [global_step, dev_summary_op, ft.loss, ft.accuracy, ft.predicted],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}, predicted {:g}".format(time_str, step, loss, accuracy, predicted))
            if writer:
                writer.add_summary(summaries, step)
            return loss, accuracy, predicted
        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        best_acc = 0.0
        best_step = 0
        predicted = 0.0
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            # 更新全局步数
            current_step = tf.train.global_step(sess, global_step)
            # 计算评估结果
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                loss_, accuracy_, predicted_ = dev_step(x_dev, y_dev, writer=dev_summary_writer)
                if accuracy_ > best_acc:
                    best_acc = accuracy_
                    predicted = predicted_
                    best_step = current_step
                    
                print("")
            # 保存模型计算结果
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

        print('\nBset dev at {}, accuray {:g}, predicted {:g}'.format(best_step, best_acc, predicted))

NameError: name 'x_train' is not defined

In [11]:
start_time = datetime.datetime.now()

In [29]:
end_time = datetime.datetime.now()
(end_time - start_time).seconds

# print('time used {:}'.format(end_time - start_time).ctime())

282

In [ ]:
time = 3600